In [1]:
import os
from PIL import Image
import numpy as np
import cv2
from commonfunctions import *
from feature_extraction import *
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import joblib
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from skimage.filters import threshold_otsu


# Preprocessing Functions

the preprocessing that we are using 
1-otsu thresholding to determine the threshold to convert our grayscale pics to binary 
2-inverting backgrounds to make all images have black background and white text
3-cropping the text from the images (increased accuracy by 1%)

In [2]:
def pre_process(img):
    thresh = threshold_otsu(img)
    binary = img > thresh
    if binary[0][0]==1:
        return np.invert(binary)
    return binary
    
def crop_image(img,tol=0):
    # img is 2D image data
    # tol  is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1).reshape(-1),mask.any(0).reshape(-1))]

# Reading the images from datasets 

In [3]:
def read_images(image_path='ACdata_base',data="test_set"):
    X = []
    y = []
    directories = [directory for directory in os.listdir(image_path) if not directory=='names.txt']
    for directory in directories:
        print(directory)
        for image_name in os.listdir(os.path.join(image_path, directory)):
            print(image_name)
            image = Image.open (os.path.join(os.path.join(image_path, directory),image_name))
            image = image.convert ("L")
            X.append(np.asarray (image ))
            y.append(directory)
    X = np.array(X)
    return X,y

In [4]:
X,y=read_images('ACdata_base')


1
0001.jpg
0002.jpg
0003.jpg
0004.jpg
0005.jpg
0006.jpg
0007.jpg
0008.jpg
0009.jpg
0010.jpg
0011.jpg
0012.jpg
0013.jpg
0014.jpg
0015.jpg
0016.jpg
0017.jpg
0018.jpg
0019.jpg
0020.jpg
0021.jpg
0022.jpg
0023.jpg
0024.jpg
0025.jpg
0026.jpg
0027.jpg
0028.jpg
0029.jpg
0030.jpg
0031.jpg
0032.jpg
0033.jpg
0034.jpg
0035.jpg
0036.jpg
0037.jpg
0038.jpg
0039.jpg
0040.jpg
0041.jpg
0042.jpg
0043.jpg
0044.jpg
0045.jpg
0046.jpg
0047.jpg
0048.jpg
0049.jpg
0050.jpg
0051.jpg
0052.jpg
0053.jpg
0054.jpg
0055.jpg
0056.jpg
0057.jpg
0058.jpg
0059.jpg
0060.jpg
0061.jpg
0062.jpg
0063.jpg
0064.jpg
0065.jpg
0066.jpg
0067.jpg
0068.jpg
0069.jpg
0070.jpg
0071.jpg
0072.jpg
0073.jpg
0074.jpg
0075.jpg
0076.jpg
0077.jpg
0078.jpg
0079.jpg
0080.jpg
0081.jpg
0082.jpg
0083.jpg
0084.jpg
0085.jpg
0086.jpg
0087.jpg
0088.jpg
0089.jpg
0090.jpg
0091.jpg
0092.jpg
0093.jpg
0094.jpg
0095.jpg
0096.jpg
0097.jpg
0098.jpg
0099.jpg
0100.jpg
0101.jpg
0102.jpg
0103.jpg
0104.jpg
0105.jpg
0106.jpg
0107.jpg
0108.jpg
0109.jpg
0110.jpg
0111.jpg

0945.jpg
0946.jpg
0947.jpg
0948.jpg
0949.jpg
0950.jpg
0951.jpg
0952.jpg
0953.jpg
0954.jpg
0955.jpg
0956.jpg
0957.jpg
0958.jpg
0959.jpg
0960.jpg
0961.jpg
0962.jpg
0963.jpg
0964.jpg
0965.jpg
0966.jpg
0967.jpg
0968.jpg
0969.jpg
0970.jpg
0971.jpg
0972.jpg
0973.jpg
0974.jpg
0975.jpg
0976.jpg
0977.jpg
0978.jpg
0979.jpg
0980.jpg
0981.jpg
0982.jpg
0983.jpg
0984.jpg
0985.jpg
0986.jpg
0987.jpg
0988.jpg
0989.jpg
0990.jpg
0991.jpg
0992.jpg
0993.jpg
0994.jpg
0995.jpg
0996.jpg
0997.jpg
0998.jpg
0999.jpg
1000.jpg
1001.jpg
1002.jpg
1003.jpg
1004.jpg
1005.jpg
1006.jpg
1007.jpg
1008.jpg
1009.jpg
1010.jpg
1011.jpg
1012.jpg
1013.jpg
1014.jpg
1015.jpg
1016.jpg
1017.jpg
1018.jpg
1019.jpg
1020.jpg
1021.jpg
1022.jpg
1023.jpg
1024.jpg
1025.jpg
1026.jpg
1027.jpg
1028.jpg
1029.jpg
1030.jpg
1031.jpg
1032.jpg
1033.jpg
1034.jpg
1035.jpg
1036.jpg
1037.jpg
1038.jpg
1039.jpg
1040.jpg
1041.jpg
1042.jpg
1043.jpg
1044.jpg
1045.jpg
1046.jpg
1047.jpg
1048.jpg
1049.jpg
1050.jpg
1051.jpg
1052.jpg
1053.jpg
1054.jpg
1055.jpg
1

<ipython-input-3-5b47d1e0e4e5>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(X)


Converting X and Y to Np arrays 

In [5]:
X=np.array(X)
y=np.array(y)

# Performing preprocssing  on the Training Data

In [7]:
pre_processed_train=[pre_process(patch.reshape(patch.shape[0],patch.shape[1],1)) for patch in X] 

In [8]:
cropped_images=[crop_image(patch) for patch in pre_processed_train]

# Performing Feature Extracting using LPQ

In [9]:
LPQ=[lpq(img.reshape(img.shape[0],img.shape[1]),winSize=11) for img in cropped_images]

# Performing Model Training 

To train our model we used for our voting classifier we used 1 NN and 1.5 SVM becuase when we tried the svm and NN before the accuracy of the SVM was higher than the neural networks 

And for our votingclassifer parameters we determined them through trial and error and these parameters gave the highest accuracy 

### First we will calculate the accuracy of the model before training it

In [12]:
clf_nn = MLPClassifier(hidden_layer_sizes=50,max_iter=3000)
clf_svm=SVC(kernel='rbf',probability=True,C=1000,gamma=5)
eclf2 = VotingClassifier(estimators=[
         ('nn', clf_nn),('svm',clf_svm)],
         voting='soft',
        flatten_transform=True,weights=[1,1.5])
print('The Accuarcy of the cross validation is ',np.mean(cross_val_score(eclf2,LPQ, y)))

The Accuarcy of the cross validation is  0.9738872403560832


### Training the Model 

In [13]:
clf_svm.probability=True  ## to print the probalilty of every class 
clf_svm.fit(LPQ, y)  ##training the model
filename = 'LPQ_BAS_FINAL.sav'
joblib.dump(clf_svm, filename)

['LPQ_BAS_FINAL.sav']

# Testing The Model with Sample Test

In [19]:
image = Image.open (os.path.join('test','7','2.png'))
image = np.array(image.convert ("L")) #loading the test 
pre_processed_image=pre_process(image) ## preprocessing
cropped_image=crop_image(pre_processed_image)
lpq_image=lpq(cropped_image,winSize=11) #perfroming the feature extraction
model=joblib.load('LPQ_BAS_FINAL.sav') #loading the model
print(model.predict_proba(lpq_image.reshape(1,-1))) # give the prob. of the right class
print(model.predict(lpq_image.reshape(1,-1))) ## give the label of the right class

[[0.0209863  0.01867594 0.02702829 0.01086858 0.00536812 0.19349805
  0.70572263 0.01663148 0.00122062]]
['7']
